# Milestone 2

Author: Berkeley Willis  
Class: Data Preparation 540-T303  
Date: 2020-04-23

### Pulling Data

First we need to pull the data, I want to do it dynamically as possible so I'll download and parse the data in frames so that we know that the data is the freshest possible

In [1]:
# Gotta pull in some basic libraries like always
import pandas as pd
import numpy as np
import re
import datetime
from bs4 import BeautifulSoup
import requests

### Setting up functions for downlaoding and parsing

Using functions for download through requests and for parsing makes things a bit simpler since it's just a large amount of copy paste code.

In [2]:
# Take XML and return parsed out dataframe
def parseIMDBData(p_soup):
    titles = []
    runtimes = []
    ratings = []
    genres = []
    imdb_ratings = []
    mcritic_scores = []
    directors = []
    votes = []
    gross = []
    years = []
    for record in p_soup.findAll("div", {"class":"lister-item"}):
        # Append data from each record
        titles.append(record.findAll("a")[1].getText())
        runtimes.append(record.find("span", {"class":"runtime"}).getText())
        # Not always a ratings
        tmp_cert = record.find("span", {"class":"certificate"})
        if tmp_cert != None:
            ratings.append(tmp_cert.getText())
        else:
            ratings.append("")
        genres.append(record.find("span", {"class":"genre"}).getText())
        imdb_ratings.append(record.find("div", {"class":"ratings-imdb-rating"}).find("strong").getText())
        tmp_mcritic = record.find("span", {"class":"metascore"})
        if tmp_mcritic != None:
            mcritic_scores.append(tmp_mcritic.getText())
        else:
            mcritic_scores.append("-1")
        votes.append(record.find("p", {"class":"sort-num_votes-visible"}).findAll("span")[1].getText())
        # Doesn't always have a grossing, else we'll default it
        tmp_gross = record.find("p", {"class":"sort-num_votes-visible"}).findAll("span")
        if len(tmp_gross) > 2:
            gross.append(tmp_gross[4].getText())
        else:
            gross.append("-1")
        years.append(record.find("span",{"class":"lister-item-year"}).getText())
        # Directors are harder to narrow down so this is a very sensitive area and might have issues if the structure
        # of the site changes drastically
        directors.append(record.findAll("p")[2].findAll("a")[0].getText())
    tmp_dict = {"title":titles,"runtime":runtimes,"rating":ratings,"genre":genres,"imdb_score":imdb_ratings,
                "metacritic_score":mcritic_scores,"director":directors,"votes":votes,"gross":gross,"year":years}
    return pd.DataFrame(tmp_dict)

# Cre1ating function to make a request and pull the data from a page
def pullIMDBData():
    # Frist URL that will be requested from
    BASE_URL = "https://www.imdb.com"
    url = BASE_URL + "/search/title/?count=100&groups=top_1000&sort=user_rating"
    # Boolean to tell let know to stop pulling
    MORE_DATA = True
    tmp_list = []
    while MORE_DATA:
        print("Pulling for " + url)
        # Get the response for the current url and parse it immediately
        soup = BeautifulSoup(requests.get(url).content, "html.parser")
        # Let's go ahead and check for and grab the next URL where applicable
        next_obj = soup.findAll("a", {"class": "lister-page-next"})
        if len(next_obj) == 0:
            MORE_DATA = False
        else:
            # Where the next search URL will be found
            url = BASE_URL + next_obj[0].get("href")
        # Now we are going to send the rest to pull the create the dataframe
        tmp_list.append(parseIMDBData(soup))
    # Now just bind them all and return
    return pd.concat(tmp_list, sort=False)

### Pull data from IMDB

Now that we have our functions setup we can downlaod and preview the data before we start cleaning operations

In [3]:
# Invoke the helper functions
imdb_movies_df = pullIMDBData()

# Display a few raw records
display(imdb_movies_df.head())

Pulling for https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=101
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=201
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=301
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=401
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=501
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=601
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=701
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=801
Pulling for https://www.imdb.com/search/title/?groups=top_1000&sort=us

,title,runtime,rating,genre,imdb_score,metacritic_score,director,votes,gross,year
0,The Shawshank Redemption,142 min,R,\nDrama,9.3,80,Frank Darabont,"2,229,168",$28.34M,(1994)
1,The Godfather,175 min,R,"\nCrime, Drama",9.2,100,Francis Ford Coppola,"1,536,551",$134.97M,(1972)
2,The Dark Knight,152 min,PG-13,"\nAction, Crime, Drama",9.0,84,Christopher Nolan,"2,202,072",$534.86M,(2008)
3,The Godfather: Part II,202 min,R,"\nCrime, Drama",9.0,90,Francis Ford Coppola,"1,075,237",$57.30M,(1974)
4,The Lord of the Rings: The Return of the King,201 min,PG-13,"\nAdventure, Drama, Fantasy",8.9,94,Peter Jackson,"1,578,347",$377.85M,(2003)


#### 1. Clean string fields

There are multiple string based fields that need to be cleaned for various uses like converting to new types, or even for the title field for merging datasets

In [4]:
# Going to re-use the function that was used previously for the same work
def sanitize_str(dirty_string):
    # Taking the titles, removing special characters, trimming whitespace, and subbing out for '_' chars
    return(re.sub(' ', '_', re.sub(' +', ' ', re.sub('[^A-Za-z0-9\s]+', '', dirty_string).strip().lower())))

# Run full sanitization on the tiel field, and run strip o nthe rest of the fields
imdb_movies_df['clean_title'] = imdb_movies_df.title.apply(sanitize_str)

imdb_movies_df['genre'] = imdb_movies_df.genre.apply(str.strip)
imdb_movies_df['rating'] = imdb_movies_df.rating.apply(str.strip)
imdb_movies_df['runtime'] = imdb_movies_df.runtime.apply(str.strip)
imdb_movies_df['imdb_score'] = imdb_movies_df.imdb_score.apply(str.strip)
imdb_movies_df['metacritic_score'] = imdb_movies_df.metacritic_score.apply(str.strip)
imdb_movies_df['director'] = imdb_movies_df.director.apply(str.strip)
imdb_movies_df['votes'] = imdb_movies_df.votes.apply(str.strip)
imdb_movies_df['gross'] = imdb_movies_df.gross.apply(str.strip)
imdb_movies_df['year'] = imdb_movies_df.year.apply(str.strip)

# Let's check on the progress of our dat ahere, should be a bit more clean
display(imdb_movies_df.head())

,title,runtime,rating,genre,imdb_score,metacritic_score,director,votes,gross,year,clean_title
0,The Shawshank Redemption,142 min,R,Drama,9.3,80,Frank Darabont,"2,229,168",$28.34M,(1994),the_shawshank_redemption
1,The Godfather,175 min,R,"Crime, Drama",9.2,100,Francis Ford Coppola,"1,536,551",$134.97M,(1972),the_godfather
2,The Dark Knight,152 min,PG-13,"Action, Crime, Drama",9.0,84,Christopher Nolan,"2,202,072",$534.86M,(2008),the_dark_knight
3,The Godfather: Part II,202 min,R,"Crime, Drama",9.0,90,Francis Ford Coppola,"1,075,237",$57.30M,(1974),the_godfather_part_ii
4,The Lord of the Rings: The Return of the King,201 min,PG-13,"Adventure, Drama, Fantasy",8.9,94,Peter Jackson,"1,578,347",$377.85M,(2003),the_lord_of_the_rings_the_return_of_the_king


#### 2. Striping strings and cleaning numbers

Now we are going to type data the the numbers

In [5]:
# Convert run time
imdb_movies_df["runtime_num"] = imdb_movies_df.runtime.str.replace(r"\D", "").astype(int)

# Converting imdb_score to float
imdb_movies_df["imdb_score"] = imdb_movies_df.imdb_score.astype(float)

# Now votes
imdb_movies_df["votes"] = imdb_movies_df.votes.str.replace(r"\D", "").astype(int)

# Remove the () marks around the years and convert it to numbers
imdb_movies_df["year"] = imdb_movies_df.year.str.replace(r"\D", "").astype(int)

# Now for gross, but we are going to convert those to raw nubers and then multiply
imdb_movies_df["gross_num"] = imdb_movies_df.gross.str.replace(r"\D", "").astype(int)
imdb_movies_df.loc[ imdb_movies_df.gross.str.contains(pat="M$"), "gross_num"] = imdb_movies_df[ imdb_movies_df.gross.str.contains(pat="M$")]["gross_num"]  * 1000000
# Setting -1 values from default now to Nan
imdb_movies_df.loc[ imdb_movies_df.gross=="-1", "gross_num"] = np.NaN

# Check on the values again
display(imdb_movies_df.head())

,title,runtime,rating,genre,imdb_score,metacritic_score,director,votes,gross,year,clean_title,runtime_num,gross_num
0,The Shawshank Redemption,142 min,R,Drama,9.3,80,Frank Darabont,2229168,$28.34M,1994,the_shawshank_redemption,142,2.834000e+09
1,The Godfather,175 min,R,"Crime, Drama",9.2,100,Francis Ford Coppola,1536551,$134.97M,1972,the_godfather,175,1.349700e+10
2,The Dark Knight,152 min,PG-13,"Action, Crime, Drama",9.0,84,Christopher Nolan,2202072,$534.86M,2008,the_dark_knight,152,5.348600e+10
3,The Godfather: Part II,202 min,R,"Crime, Drama",9.0,90,Francis Ford Coppola,1075237,$57.30M,1974,the_godfather_part_ii,202,5.730000e+09
4,The Lord of the Rings: The Return of the King,201 min,PG-13,"Adventure, Drama, Fantasy",8.9,94,Peter Jackson,1578347,$377.85M,2003,the_lord_of_the_rings_the_return_of_the_king,201,3.778500e+10


#### 3. Look for and drop any non-unique titles

The last thing we need on a merge is a cartesian product that would create more duplicates, so the bes idea here is to remove the duplicates when possible.

In [6]:
# First we should look and see if there are any duplicates,so that we can 
display(imdb_movies_df[ imdb_movies_df.title.isin(imdb_movies_df.title[ imdb_movies_df.title.duplicated()]) ].head())

# Some thing that I see here is that yes the movie titles are the same but the directors are different
# likely due to international differences so I will take that into account and only de-dup on combinations 

print("Currently have {} records before de-dup".format(imdb_movies_df.shape[0]))
imdb_movies_df = imdb_movies_df.drop_duplicates(subset=["title","director"])
print("Left with {} records after de-dup".format(imdb_movies_df.shape[0]))

# So in this case there were no duplicates in this case so hopefully it should be pretty easy to merge

,title,runtime,rating,genre,imdb_score,metacritic_score,director,votes,gross,year,clean_title,runtime_num,gross_num
59,Drishyam,160 min,Not Rated,"Crime, Drama, Thriller",8.4,-1,Jeethu Joseph,28855,-1,2013,drishyam,160,NaN
33,Drishyam,163 min,Not Rated,"Crime, Drama, Mystery",8.2,-1,Nishikant Kamat,62172,$0.74M,2015,drishyam,163,7.400000e+07
91,The Girl with the Dragon Tattoo,158 min,R,"Crime, Drama, Mystery",7.8,71,David Fincher,405031,$102.52M,2011,the_girl_with_the_dragon_tattoo,158,1.025200e+10
97,The Girl with the Dragon Tattoo,152 min,R,"Crime, Drama, Mystery",7.8,76,Niels Arden Oplev,203945,$10.10M,2009,the_girl_with_the_dragon_tattoo,152,1.010000e+09


Currently have 1000 records before de-dup
Left with 1000 records after de-dup


#### 4. Add some extra fields that identify certain trains that might be helpful

We can also go eahd and marks somethings like a relatively high metacritic score or certain positions in the rankings here. At least need to add rank as index

In [7]:
# Gotta add rankings for each one
imdb_movies_df['ranking'] = range(1,imdb_movies_df.shape[0]+1)

imdb_movies_df['top100'] = False
imdb_movies_df.loc[ (imdb_movies_df.ranking<=100) ,'top100'] = True

# I wouldn't surprised if most of these top movies were dramas so we'll also create a new column for it
# Could mine for this during analysis but sometimes these preparation steps can make it easier later
imdb_movies_df['is_drama'] = False
imdb_movies_df['is_crime'] = False
imdb_movies_df.loc[imdb_movies_df.genre.str.contains(pat="Drama"), "is_drama"] = True
imdb_movies_df.loc[imdb_movies_df.genre.str.contains(pat="Crime"), "is_crime"] = True

# Take one last look at what we've got so far
display(imdb_movies_df.head())

,title,runtime,rating,genre,imdb_score,metacritic_score,director,votes,gross,year,clean_title,runtime_num,gross_num,ranking,top100,is_drama,is_crime
0,The Shawshank Redemption,142 min,R,Drama,9.3,80,Frank Darabont,2229168,$28.34M,1994,the_shawshank_redemption,142,2.834000e+09,1,True,True,False
1,The Godfather,175 min,R,"Crime, Drama",9.2,100,Francis Ford Coppola,1536551,$134.97M,1972,the_godfather,175,1.349700e+10,2,True,True,True
2,The Dark Knight,152 min,PG-13,"Action, Crime, Drama",9.0,84,Christopher Nolan,2202072,$534.86M,2008,the_dark_knight,152,5.348600e+10,3,True,True,True
3,The Godfather: Part II,202 min,R,"Crime, Drama",9.0,90,Francis Ford Coppola,1075237,$57.30M,1974,the_godfather_part_ii,202,5.730000e+09,4,True,True,True
4,The Lord of the Rings: The Return of the King,201 min,PG-13,"Adventure, Drama, Fantasy",8.9,94,Peter Jackson,1578347,$377.85M,2003,the_lord_of_the_rings_the_return_of_the_king,201,3.778500e+10,5,True,True,False


#### 5. Last finalize the dataframe

Lastly it would be helpful to drop unnecessary columns, and rename those to make sure they don't get confused during the merge with other datasets. 

In [8]:
# Finally we've some of the raw columns that have repatitive and un prepared data, so now I am going to
# limit the number of columns and rename them
final_imdb_movies_df = imdb_movies_df[["ranking","title","director","clean_title","rating","genre","imdb_score",
                                      "metacritic_score","votes","year","runtime_num","gross_num","top100",
                                      "is_drama","is_crime"]]

final_imdb_movies_df.columns = ["imdb_ranking","imdb_title","imdb_director","clean_title","imdb_rating",
                                "imdb_genres","imdb_score","metacritic_score","imdb_votes","imdb_year",
                                "imdb_runtime","imdb_gross","imdb_top100","imdb_drama","imdb_crime"]

display(final_imdb_movies_df.head())

,imdb_ranking,imdb_title,imdb_director,clean_title,imdb_rating,imdb_genres,imdb_score,metacritic_score,imdb_votes,imdb_year,imdb_runtime,imdb_gross,imdb_top100,imdb_drama,imdb_crime
0,1,The Shawshank Redemption,Frank Darabont,the_shawshank_redemption,R,Drama,9.3,80,2229168,1994,142,2.834000e+09,True,True,False
1,2,The Godfather,Francis Ford Coppola,the_godfather,R,"Crime, Drama",9.2,100,1536551,1972,175,1.349700e+10,True,True,True
2,3,The Dark Knight,Christopher Nolan,the_dark_knight,PG-13,"Action, Crime, Drama",9.0,84,2202072,2008,152,5.348600e+10,True,True,True
3,4,The Godfather: Part II,Francis Ford Coppola,the_godfather_part_ii,R,"Crime, Drama",9.0,90,1075237,1974,202,5.730000e+09,True,True,True
4,5,The Lord of the Rings: The Return of the King,Peter Jackson,the_lord_of_the_rings_the_return_of_the_king,PG-13,"Adventure, Drama, Fantasy",8.9,94,1578347,2003,201,3.778500e+10,True,True,False
